In [1]:
import agents_env
from agents_env.agent_replay_motion import HumanoidReplay
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *

In [2]:

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap

In [3]:
envs.register_environment('humanoidReplay', HumanoidReplay)

In [4]:
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'

In [11]:
#data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = move_only_arm(jp.asarray(trajectory.data_pos))
data_vel_mocap = move_only_arm_vel(jp.asarray(trajectory.data_vel))

data_dict_mocap = trajectory.duration_dict

In [10]:
#data_vel_mocap[-1]

In [12]:
env_name = 'humanoidReplay'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)

In [13]:

def visualizer(rollout):
    media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

In [14]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]

for k in range(len(env.dict_duration)):
    qpos = state.pipeline_state.qpos
    tmp_pos = env.reference_trajectory_qpos[k]
    updated = qpos.at[:].set(tmp_pos[:])
    
    state = state.tree_replace({'pipeline_state.qpos': updated})
    ctrl = -0.1 * jp.ones(env.sys.nv)
    state = jit_step(state, ctrl)
    #print(state.pipeline_state.time)
    rollout.append(state.pipeline_state)

In [15]:
visualizer(rollout)

This browser does not support the video tag.


In [14]:
env.sys.qpos0

Array([0. , 0. , 0.9, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], dtype=float32)